In [1]:
### General imports ###
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from time import sleep
import re
import os
import argparse
from collections import OrderedDict
import matplotlib.animation as animation

### Image processing ###
from scipy.ndimage import zoom
from scipy.spatial import distance
import imutils
from scipy import ndimage
import cv2
import dlib
# from __future__ import division
from imutils import face_utils

### CNN models ###
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, SeparableConv2D
from keras.utils import np_utils
from keras.regularizers import l2#, activity_l2
from keras.optimizers import SGD, RMSprop
from keras.utils import to_categorical
from keras.layers.normalization import BatchNormalization
from keras import models
from keras.utils.vis_utils import plot_model
from keras.layers import Input, GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras import layers

### Build SVM models ###
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm

### Same trained models ###
import h5py
from keras.models import model_from_json
import pickle

Using TensorFlow backend.


In [2]:
shape_x = 48
shape_y = 48

def detect_face(frame):
    
    #Cascade classifier pre-trained model
    cascPath = './haarcascade_frontalface_default.xml'
    faceCascade = cv2.CascadeClassifier(cascPath)
    
    #BGR -> Gray conversion
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #Cascade MultiScale classifier
    detected_faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=6,
                                                  minSize=(shape_x, shape_y),
                                                  flags=cv2.CASCADE_SCALE_IMAGE)
    coord = []
    
    for x, y, w, h in detected_faces :
        if w > 100 :
            sub_img=frame[y:y+h,x:x+w]
            #cv2.rectangle(frame,(x,y),(x+w,y+h),(0, 255,255),1)
            coord.append([x,y,w,h])
    
    return gray, detected_faces, coord

#Extraire les features faciales
def extract_face_features(faces, offset_coefficients=(0.075, 0.05)):
    gray = faces[0]
    detected_face = faces[1]
    
    new_face = []
    
    for det in detected_face :
        #Region dans laquelle la face est détectée
        x, y, w, h = det
        #X et y correspondent à la conversion en gris par gray, et w, h correspondent à la hauteur/largeur
    
        #Offset coefficient, np.floor takes the lowest integer (delete border of the image)
        horizontal_offset = np.int(np.floor(offset_coefficients[0] * w))
        vertical_offset = np.int(np.floor(offset_coefficients[1] * h))

        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #gray transforme l'image
        extracted_face = gray[y+vertical_offset:y+h, x+horizontal_offset:x-horizontal_offset+w]
    
        #Zoom sur la face extraite
        new_extracted_face = zoom(extracted_face, (shape_x / extracted_face.shape[0],shape_y / extracted_face.shape[1]))
        #cast type float
        new_extracted_face = new_extracted_face.astype(np.float32)
        #scale
        new_extracted_face /= float(new_extracted_face.max())
        #print(new_extracted_face)
    
        new_face.append(new_extracted_face)
    
    return new_face

In [3]:
from tensorflow import keras
model = keras.models.load_model('./model[ekaggle].h5')

In [4]:
model.summary()

In [5]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [6]:
thresh = 0.25
frame_check = 20
face_detect = dlib.get_frontal_face_detector()
predictor_landmarks = dlib.shape_predictor("./face_landmarks.dat")

In [7]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [8]:
(nStart, nEnd) = face_utils.FACIAL_LANDMARKS_IDXS["nose"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]
(jStart, jEnd) = face_utils.FACIAL_LANDMARKS_IDXS["jaw"]

(eblStart, eblEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eyebrow"]
(ebrStart, ebrEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eyebrow"]

In [9]:
desiredLeftEye=(0.35, 0.35)

def align(gray, rect):
    # convert the landmark (x, y)-coordinates to a NumPy array
    shape = predictor(gray, rect)
    shape = shape_to_np(shape)
 
    # extract the left and right eye (x, y)-coordinates
    (lStart, lEnd) = FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = FACIAL_LANDMARKS_IDXS["right_eye"]
    leftEyePts = shape[lStart:lEnd]
    rightEyePts = shape[rStart:rEnd]
        
    # compute the center of mass for each eye
    leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
    rightEyeCenter = rightEyePts.mean(axis=0).astype("int")
 
    # compute the angle between the eye centroids
    dY = rightEyeCenter[1] - leftEyeCenter[1]
    dX = rightEyeCenter[0] - leftEyeCenter[0]
    angle = np.degrees(np.arctan2(dY, dX)) - 180

    desiredRightEyeX = 1.0 - desiredLeftEye[0]
 

    dist = np.sqrt((dX ** 2) + (dY ** 2))
    desiredDist = (desiredRightEyeX - desiredLeftEye[0])
    desiredDist *= self.desiredFaceWidth
    scale = desiredDist / dist
        
    # compute center (x, y)-coordinates (i.e., the median point)
    # between the two eyes in the input image
    eyesCenter = ((leftEyeCenter[0] + rightEyeCenter[0]) // 2,
            (leftEyeCenter[1] + rightEyeCenter[1]) // 2)
 
    # grab the rotation matrix for rotating and scaling the face
    M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)
 
    # update the translation component of the matrix
    tX = self.desiredFaceWidth * 0.5
    tY = self.desiredFaceHeight * self.desiredLeftEye[1]
    M[0, 2] += (tX - eyesCenter[0])
    M[1, 2] += (tY - eyesCenter[1])
        
    # apply the affine transformation
    (w, h) = (self.desiredFaceWidth, self.desiredFaceHeight)
    #output = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)
    output = cv2.warpAffine(gray, M, (w, h), flags=cv2.INTER_CUBIC)
    # return the aligned face
    return output

In [10]:
import cv2

In [11]:
#Lancer la capture video

COLOR_PRED=(0,0,255)
video_capture = cv2.VideoCapture(0)
flag = 0
j = 1

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    
    face_index = 0
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = face_detect(gray, 1)
    #gray, detected_faces, coord = detect_face(frame)
    
    for (i, rect) in enumerate(rects):
        
        shape = predictor_landmarks(gray, rect)
        shape = face_utils.shape_to_np(shape)
    
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        face = gray[y:y+h,x:x+w]
        
        #Zoom sur la face extraite
        face = zoom(face, (shape_x / face.shape[0],shape_y / face.shape[1]))
        #cast type float
        face = face.astype(np.float32)
        #scale
        face /= float(face.max())
        face = np.reshape(face.flatten(), (1, 48, 48, 1))
        prediction = model.predict(face)
        prediction_result = np.argmax(prediction)
        
        # Rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
    
        cv2.putText(frame, "Face #{}".format(i + 1), (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.rectangle(frame,(20,100),(220,280),(255,255,255,0.5),-1)

        
        
         # 12. Add prediction probabilities
        cv2.putText(frame, "Emotional report : ",(40,120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PRED, 0)
        cv2.putText(frame, "Angry : " + str(round(prediction[0][0],3)),(40,140), cv2.FONT_HERSHEY_SIMPLEX, 0.5,COLOR_PRED, 0)
        cv2.putText(frame, "Disgust : " + str(round(prediction[0][1],3)),(40,160), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PRED, 0)
        cv2.putText(frame, "Fear : " + str(round(prediction[0][2],3)),(40,180), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PRED, 1)
        cv2.putText(frame, "Happy : " + str(round(prediction[0][3],3)),(40,200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PRED, 1)
        cv2.putText(frame, "Sad : " + str(round(prediction[0][4],3)),(40,220), cv2.FONT_HERSHEY_SIMPLEX, 0.5,COLOR_PRED, 1)
        cv2.putText(frame, "Surprise : " + str(round(prediction[0][5],3)),(40,240), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PRED, 1)
        cv2.putText(frame, "Neutral : " + str(round(prediction[0][6],3)),(40,260), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_PRED, 1)
        

                   
        if prediction_result == 0 :
            cv2.putText(frame, "Angry : " + str(round((prediction[0][0])*100,3))+" %",(x,y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 255, 0), 2)
#             cv2.putText(frame, "Angry",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif prediction_result == 1 :
            cv2.putText(frame, "Disgust : " + str(round((prediction[0][0])*100,3))+" %",(x,y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 255, 0), 2)
        elif prediction_result == 2 :
            cv2.putText(frame, "Fear : " + str(round((prediction[0][0])*100,3))+" %",(x,y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 255, 0), 2)
        elif prediction_result == 3 :
            cv2.putText(frame, "Happy : " + str(round((prediction[0][3])*100,3))+" %",(x,y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 255, 0), 2)
        elif prediction_result == 4 :
            cv2.putText(frame, "Sad : " + str(round((prediction[0][0])*100,3))+" %",(x,y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 255, 0), 2)
        elif prediction_result == 5 :
            cv2.putText(frame, "Surprise : " + str(round((prediction[0][0])*100,3))+" %",(x,y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 255, 0), 2)
        else :
            cv2.putText(frame, "Neutral : " + str(round((prediction[0][0])*100,3))+" %",(x,y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 255, 0), 2)
                   
       
        cv2.putText(frame,'Number of Faces : ' + str(i+1),(40, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, COLOR_PRED, 4)
        j = j + 1
        cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()